<a href="https://colab.research.google.com/github/kameda-yoshinari/DataAlgo-UT/blob/main/DataAlgo_UT(013)_NumberPlace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 6.2.3. ナンバープレース (Number place)

ナンバープレースは 数独 Sudoku とも呼ばれるペンシルパズルの一つである．本問題がバックトラック法によって解けることを示す．


**いつもの約束**  
１つのコードセルだけの実行は Ctrl + Enter．  
エディタで「インデント幅（スペース）は4で表示」「行番号を表示」「インデントガイドを表示」．  
内部では日本語はUTF-8で表現されている．


# 準備

インスタンスに接続し起動する．  
下記の手順でGoogle Driveをマウントする．  
マウント先に移動し，作業フォルダとする．  
これによって，インスタンスがリセットされてもGoogle Drive内にファイルが保存されるようにする．

In [ ]:
!echo "Google Driveをマウントします"
from google.colab import drive 
drive.mount('/content/drive')

In [ ]:
!echo "今回の作業用フォルダを作成しそこに移動します"
%cd /content/drive/My\ Drive/
%mkdir -p 202104_DataAlgo/DA2021_013
%cd       202104_DataAlgo/DA2021_013
!ls
!echo "日本時間表示"
!rm /etc/localtime
!ln -s /usr/share/zoneinfo/Japan /etc/localtime
!date

# ナンバープレース

**問題**

数字の1から9を使う．9x9の盤を，それぞれの数字を9回使って全81マスを埋める問題である．ただし，以下の3条件を全て満たす数の配置をナンバープレースの解とする．

* 各列は1から9を1度ずつのみ用いて埋める．
* 各行は1から9を1度ずつのみ用いて埋める．
* 3x3の9つの小ブロックに分け，各小ブロックも1から9を1度ずつ用いて埋める．

明らかに，これを満たす解は（数が多いとはいえ）有限である．  
厳密な解の数え上げは本授業では扱わない．

> https://ja.wikipedia.org/wiki/Sudoku  
> https://en.wikipedia.org/wiki/Sudoku (英語版の方が詳述されている)

**人間向けの解法**

パズルとしては81マスのうち30マス程度がすでに埋まっている．つまり，残りの空マスに入れるべき数字を探索するのが解の探索となる．

ナンバープレースの解法については，巷でたくさんの方法が紹介されているが，これらはいずれも「人間の頭の特性」に合わせた解法である．そのため，もし，ナンバープレースを自動的に解くプログラムを作ろうと思うのなら，**これらの解法に従うのは愚策**である．

例えば，多くの人間にとって，同時に複数の解候補を正確に覚え続けていることは大変困難である．また，複数の解候補を瞬時に切替ながら評価することも苦手である．ナンバープレースで上級と呼ばれる問題では，解の探索のごく初期から複数の選択肢を同時に考え続けなくてはいけないようにしてあったりする．こうしたことは人間は苦手であるが，コンピュータにとっては何の問題もない．

実際，あとで確認できることであるが，バックトラック法のプログラムによってナンバープレースの問題を解かせると，初級問題も上級問題もほぼ何の違いもなく解を得る．

計算量の観点から考えるならば，空マスが多いほど解候補が増えて計算量が増えることが予想される．ところが，人間向けの初級問題から上級問題については，上級だからといって空マスが多いわけではない．このことからも，「初級」「上級」というのはあくまで人間の頭の特性に対してであって，アルゴリズム論的な意味ではないことが伺える．

なお，余談ではあるが，パズルとしては解が1つのみになるよう調節してあることがほとんどである．




# 全解探索法

**案A**

3条件のうちの1つのみを解候補の生成に用いるとすると，解候補の数は (9!)<sup>9</sup>と考えることができる．残りの2条件の確認には，2 x 9 x 9 程度必要であろうから，結局，全体としての計算コストは (9!)<sup>9</sup> x 2 x 9<sup>2</sup> 程度になる．（ここでは問題のサイズが固定なのでビッグオー表現を用いていないことに注意する．）

実際には，すでに埋まっているマスでは数字の選択を考慮する必要がないから，これよりは（ずっと）計算量が少なくて済むことが予想される．ただし，それでも現実的に行える解候補の数ではない．

**案B**

パズルとしては81マスのうち30マス程度がすでに埋まっている．空マス数をKとすると，その解候補は 9<sup>K</sup> だけあると考えることもできる．

**案C**

3条件を満たす解の数は多いとはいえ有限である．ナンバープレースの問題の解答はふつう1通りである．つまり，もし全ての解をメモリ上に展開しておけるなら，ナンバープレースの数字が埋まっている部分の情報を検索キーとして，単なる検索で解を瞬時に得ることができると考えられる．

しかし，この実現には相当な（非現実的な）量のメモリ（空間計算量）が必要となる．英語版wikipediaを参照すること．


# バックトラック法

ナンバープレースをバックトラック法で解くのは実はかなり直接的である．解の条件を確認する部分こそ新しく考える必要があるが，あとはこれまでの Knight tour や N Queens とほぼ同じアルゴリズムで実現できる．

１つの空マスについて入れるべき数字を吟味するとき，満たすべき条件は3つある．すなわち，その列の拘束条件，その行の拘束条件，それにその空マスを含む3x3小ブロックの拘束条件である．

つまり，全ての空マスについて，順にこの3条件が満たすような数字を埋めて行けばよい．

ここで注意すべきは，（人間向けの解法と違って）どの空マスから処理するかは全く気にしていないことである．バックトラック法とはいえ，そのベースは全解探索法なので，最終的には全ての空マスを処理する以上，プログラムとして空マスを並べ替えることに意味はない．一応，手を付ける空マスの順序を（人間のするように適宜）入れ替えると，最初の解が見つかるまでの時間が早くなる可能性はある．しかし，あとで実行してみればわかるが，この程度の問題であれば，そのようなことにプログラミングの労力をかけるだけ無駄である．







# バックトラック法によるCプログラム

**目標**

ナンバープレースの解をバックトラック法で解くプログラムを作成する．問題はテキストファイル（出題ファイル）で与えるものとする．

**説明**

これまでの他のバックトラックプログラムと同じく，深さ優先探索を用いる．その実装には再帰呼出を用いる．

バックトラック法では，再帰呼出によって先の探索に進んだ後で，戻ってきたときに現状復帰する必要があることに注意する．

105-108行が行き順における書き込みに相当し，112-115行が戻り時の原状復元に相当する．

出題ファイル読み込み時に空セルを celltoexamine[] 配列に記録しておく．解探索時にはこの空セルを順に処理していく．

**コード**

x (x=1, ... , 9) の縦方向制約を示すのにcheckx[x][], 同じく y の横方向制約を示すのにchecky[y][]を用いる．3x3小ブロックbの制約を表すのにcheckb[b][]を用いる．ここで，b = (x / 3) + (y / 3) * 3 とする．

**備考**

実行実時間の計測に，Unix (Linux) でよく使われる gettimeofday()関数を利用している．
参考にされたい．gettimeofday()関数は現実世界の時刻を取得するので，timeコマンドのように内訳を調べることはできない．




In [ ]:
%%writefile NumberPlace_J.c 
// Number Place, a.k.a. Sudoku by backtrack method
//  kameda[at]ccs.tsukuba.ac.jp, 2020.
#include <stdio.h>
#include <stdlib.h> // atoi()
#include <string.h> // strcmp()
#include <sys/time.h> // gettimeofday()

#define N 9 // Sudoku board size

typedef struct {int x; int y;} vec2i; // 2D coordinates

int board[N][N];       // 0 ... undertermined, 1-9 ... answers
int numlockedcell = 0; // number of locked cells (given as the problem)

vec2i celltoexamine[N*N]; // List of empty cells in the problem
int numemptycell = 0;     // Number of empty cells. numlockedcell + numemptycell = N * N

// three kinds of the conditions. binary representation.
int checkx[N][N]; // if figure f(1-9) has been already used on line-x,  checky[x][f-1] = 1, 0 otherwise
int checky[N][N]; // if figure f(1-9) has been already used on line-y,  checky[y][f-1] = 1, 0 otherwise
int checkb[N][N]; // if figure f(1-9) has been already used on block-b, checkb[b][f-1] = 1, 0 otherwise

int num_answer = 0; // Number of the answers (expected to be 1 for Sudoku)
int verboselevel = 0; // Verbose level (0 minimum, 1 show answers, 2 show boards)


// Return blok-ID from (x,y) coord
int xy2b(int x, int y) {
    return ((x / 3) + (y / 3) * 3); // Based on the C language syntax
}

// Show board
//   Empty cell is denoted by '.'
void showboard(void){
    vec2i pt;

    for (pt.y = 0; pt.y < N; pt.y++) {
        printf("%d ", pt.y);
        for (pt.x = 0; pt.x < N; pt.x++) {
            if (board[pt.x][pt.y] != 0) {
                printf("%d", board[pt.x][pt.y]);
            } else {
                printf(".");
            }
        }
        printf("\n");
    }
    return ;
}

//  Read the board (the problem) info from a file.
//    If "-" is specified, read the dat from stdin.
int readboard(char *filename){
    FILE *fd;
    vec2i pt;
	
    if (strcmp(filename, "-") == 0) {
        fd = stdin;
    } else if ((fd = fopen(filename, "r")) == NULL) {
        printf("readboard: cannnot open %s\n", filename);
        return -1;
    }

    for (pt.y = 0; pt.y < N; pt.y++) {
        for (pt.x = 0; pt.x < N; pt.x++) {
            if (fscanf(fd, "%d", &(board[pt.x][pt.y])) != 1) {
                printf("readboard: error at (x,y) = (%d, %d)\n", pt.x, pt.y);
	            return -2;
            }
            if (board[pt.x][pt.y] < 1 || board[pt.x][pt.y] > 9) {
                board[pt.x][pt.y] = 0; // cleaning up to zero / foolproof
            }
            if (board[pt.x][pt.y] == 0) {
                celltoexamine[numemptycell++] = pt; // Append this empty cell to the list
            }
        }
    }
    fclose(fd);

    showboard();
    printf("readboard: empty cells = %d\n", numemptycell);

    return 0;
}

// Find the right figure (1-9) at pos-th empty cells
void sudokuonestep(int pos){
    int x, y, b;
    int f;
	
    // Reach to an answer
    if (pos == numemptycell) {
        if (verboselevel >= 1) printf("======== %d ========\n", num_answer);
        if (verboselevel >= 2) showboard();
        num_answer++;
        return ;
    }
	
    x = celltoexamine[pos].x;
    y = celltoexamine[pos].y;
    b = xy2b(x, y);
    for (f = 1; f <= N; f++) {
        if (checkx[x][f-1] == 0 && checky[y][f-1] == 0 && checkb[b][f-1] == 0) {
            checkx[x][f-1] = 1; // Mark
            checky[y][f-1] = 1; // Mark
            checkb[b][f-1] = 1; // Mark
            board[x][y] = f;    // Record
            
            sudokuonestep(pos+1);
            
            checkx[x][f-1] = 0; // Unmark
            checky[y][f-1] = 0; // Unmark
            checkb[b][f-1] = 0; // Unmark
            board[x][y] = 0;    // Clear
        }
    }
}

// Main function
int main(int argc, char *argv[]){
    int r = 0;
    int i, f;
    int x, y, b;
    struct timeval ts, te;

    // examine options
    if (argc != 3) {
        printf("This command needs Problem-file and Verbose-Level.\n");
        printf("Please set a problem file and verbose level (0 or upper).\n");
        return -1;
    }

    // Read file
    if ((r = readboard(argv[1])) != 0) {
        printf("Error: file reading status : %d\n", r);
        return r;
    }

    // verbose level
    verboselevel = atoi(argv[2]);

    // Initialization of the conditions
    for (i = 0; i < N; i++) {
        for (f = 0; f < N; f++) {
            checkx[i][f] = checky[i][f] = checkb[i][f] = 0;
        }
    }

    // Mark the figures that are set in the problem
    for (x = 0; x < N; x++) {
        for (y = 0; y < N; y++) {
            if (board[x][y] != 0) {
                b = xy2b(x, y);
	            checkx[x][board[x][y]-1] = checky[y][board[x][y]-1] = checkb[b][board[x][y]-1] = 1;
            }
        }
    }

    // Go	
    gettimeofday(&ts, NULL);
    sudokuonestep(0);
    gettimeofday(&te, NULL);

    // Show the result
    printf("===================\n");
    printf("Number of the answers: %d\n", num_answer);
    printf("Time = %.6f [sec]\n", (float)(te.tv_sec - ts.tv_sec) + (te.tv_usec - ts.tv_usec) / 1000000.0);

    return 0;
}



コンパイルしてエラーが無いことを確認．

In [ ]:
!gcc -Wall -o NumberPlace_J NumberPlace_J.c

下記の実行をする前に，１つ下の「出題ファイル」にて p5252.txt を保存しておくこと．
p5252は [出題サイト SUDOKU](https://www.sudoku.name/index-jp.php) によれば「上級++」で相当に難しい問題のようである．（私はsudokuを全くしないので実際に人間としてどう感じるべきなのかわからない）
http://www.sudoku.name/puzzles/jp/5252/f0fcf351df4eb6786e9bb6fc4e2dee02#5252

現在のCPUならば，実行時間は1秒かからない．knight tourなどと違って，最適化をかける必要すら感じない実行速度である．

ここで注意してほしいのは，このプログラムでは空マスを左上から順に埋めているだけであるということである．人間には耐えられないアルゴリズムであるが，プログラムの記述としてはむしろこのほうが簡潔で扱いやすい．（もし人間向けの解法を実装したら相当に面倒くさいプログラムになる上に，必ず解に辿り着ける保証を与えられるかどうかも怪しいであろう）

In [ ]:
!time ./NumberPlace_J p5252.txt 2

# 出題ファイル

出題ファイルはテキストで用意する．
基本的には，初めの9行に9個の数字をおいて問題を記述する．
空欄は 0 で表現する．数字の間のスペースは1つ以上なら幾つでもよい．

なお，NumberPlace_j プログラムを読めば分かるが，実はfscanfで81個の整数を読んでいるだけで，それを終えた時点で読み込みは終了する．言い換えれば，途中に空行があってもかまわないし，改行があってもなくても構わない．81個の数字のあとに何を書いても問題の読み込みには関係ない．


In [ ]:
%%writefile p5252.txt
0 2 0  9 0 0  0 0 6
1 0 0  0 0 0  0 9 0
0 0 8  0 0 7  0 0 0

5 0 0  0 0 0  7 0 0
0 0 7  8 0 0  1 0 0
0 0 9  4 0 0  0 0 2

0 0 0  2 0 0  4 0 0
0 6 0  0 0 0  0 0 3
3 0 0  0 0 5  0 8 0

Taken from:
http://www.sudoku.name/puzzles/jp/5252/f0fcf351df4eb6786e9bb6fc4e2dee02#5252
上級++ (非常に難しい問題のようである)

実行結果例

0 .2.9....6
1 1......9.
2 ..8..7...
3 5.....7..
4 ..78..1..
5 ..94....2
6 ...2..4..
7 .6......3
8 3....5.8.
readboard: empty cells = 59
======== 0 ========
0 423918576
1 176523894
2 958647321
3 512396748
4 647852139
5 839471652
6 781239465
7 265784913
8 394165287
===================
Number of the answers: 1
Time = 0.029590 [sec]


In [ ]:
%%writefile p22833.txt
0 0 0  0 9 0  0 0 0
0 8 9  0 0 0  3 4 0
7 0 0  8 0 3  0 0 2

0 1 0  0 0 0  0 6 0
0 0 8  3 0 4  7 0 0
0 7 0  0 0 0  0 5 0

8 0 0  5 0 7  0 0 1
0 5 4  0 0 0  9 7 0
0 0 0  0 2 0  0 0 0

https://www.sudoku.name/puzzles/jp/22833/42fe880812925e520249e808937738d2
#22833
上級++


In [ ]:
%%writefile p22833-cut1.txt
0 0 0  0 9 0  0 0 0
0 8 9  0 0 0  3 4 0
7 0 0  8 0 3  0 0 2

0 1 0  0 0 0  0 6 0
0 0 8  3 0 4  7 0 0
0 7 0  0 0 0  0 5 0

8 0 0  5 0 7  0 0 0
0 5 4  0 0 0  9 7 0
0 0 0  0 2 0  0 0 0

https://www.sudoku.name/puzzles/jp/22833/42fe880812925e520249e808937738d2
#22833 
上級++
(9,7)の"1"を削除


In [ ]:
%%writefile p22833-cut4.txt
0 0 0  0 9 0  0 0 0
0 8 9  0 0 0  3 4 0
7 0 0  8 0 3  0 0 2

0 1 0  0 0 0  0 6 0
0 0 8  3 0 4  7 0 0
0 7 0  0 0 0  0 5 0

0 0 0  0 0 0  0 0 0
0 5 4  0 0 0  9 7 0
0 0 0  0 2 0  0 0 0

https://www.sudoku.name/puzzles/jp/22833/42fe880812925e520249e808937738d2
#22833 
上級++
7行目を全て(数字4つ)削除


あとは同じように自分で解かせてみたい問題に対応する出題ファイルを用意して実行してみるとよい．

試してみればわかることだが，実行時間は初級か上級かに全く関係ない．

# 節末課題

1. 問題の難易度と実行時間との関係  
出題サイト等に行って，人間向けの難易度の異なる問題を3つ以上用意し，それぞれで実行時間を計測して，問題の難易度と実行時間との関係を考察せよ．そのような関係になる理由についても考察すること．


2. 解の一意性  
幾つかの出題において，指定されていた数字を1つ以上消去することで，解が一意でなくなるかどうか確認してみよ．NumberPlace_Jプログラムを利用して試行する際に，最初のうちは verbose level を 0 にすること．(指数爆発する可能性がある．)


3. 関数xy2b()  
NumberPlace_Jプログラムにおいて，空マス(x,y)の属する 3x3小ブロックの ID が xy2b()によって決定できることを解説せよ．C言語における演算の特性にも言及すること．必要があれば適宜図示すること．


4. X-SUDOKU  
2本の対角線においても，1-9の数字を1つずつ配置するという制約を増やした問題をX-SUDOKUという．例えば [Puzzle madness](https://puzzlemadness.co.uk/xsudoku) などに出題がある．これを解くプログラムを作成せよ．





# 出典

筑波大学工学システム学類  
データ構造とアルゴリズム  
担当：亀田能成  
2020/06/09 初版